In [1]:
import k3d
import numpy as np
from scipy import stats
import scipy.optimize as optimize

import sys
sys.path.insert(0, "..")

from Geometry.SM import SM

In [2]:
# Simulate a survey of a supermodule

# Define the survey room as a container of the supermodule. Convenient to define the room itself as a supermodule.

class Room(SM):
    # define a room containing a single supermodule

    # select the supermodule type to be used in the survey by setting the index
    super_module = {0:'bottom', 1:'barrel', 2:'top'}[0]

    room_sms = []
    # one supermodule in the room - its position and orientation are not well known
    room_sms.append({'name': super_module, 'kind': super_module,
                    'loc': [0., 0., 0.],
                    'loc_sig': [100.0, 100.0, 30.0],
                    'rot_axes': 'ZYX',
                    'rot_angles': [0., 0., 0.],
                    'rot_angles_sig': [0.02, 0.02, 0.04]})

    SM.devices_design['room'] = room_sms
    
    # exaggerate the standard deviation of the placements of mpmts:
    loc_sig = 3.0 # mm
    rot_angle_sig = 0.02 # radians
    
    for mpmt in {'bottom': SM.bottom_mpmts, 'barrel':SM.barrel_mpmts, 'top':SM.top_mpmts}[super_module]:
        mpmt['loc_sig'] = [loc_sig]*3
        mpmt['rot_angles_sig'] = [rot_angle_sig]*3


In [3]:
# create the supermodule and place it in the room at a random location:

my_room = Room('my_room', kind='room', device_type=SM)

In [4]:
# get the supermodule that is in the room 

sm = my_room.sms[0]

# look at RMS of deviations of the true fiduical markers compared to design positions:
sum2 = 0.
for mpmt in sm.mpmts:
    fiducials_true = mpmt.get_fiducials('true')
    fiducials_design = mpmt.get_fiducials('design', sm) # we don't know the location of the SM in the room
    for i in range(4):
        diff = np.subtract(fiducials_true[i],fiducials_design[i])
        sum2 += np.dot(diff,diff)
rms = np.sqrt(sum2/(4*len(sm.mpmts)))
print('RMS of deviations of the true fiduical markers compared to design positions: %.2f mm' % rms)

RMS of deviations of the true fiduical markers compared to design positions: 77.77 mm


In [5]:
print(sm.mpmts[0].get_fiducials('true'))
print(sm.mpmts[0].get_fiducials('design', sm))

[array([ 173.32740244, -243.12260092,  -79.3052958 ]), array([-219.69019963, -233.81058539,  -74.28170822]), array([182.61432381, 149.92253822, -81.31956777]), array([-210.40327826,  159.23455375,  -76.2959802 ])]
[array([ 196.58, -196.58,  -30.  ]), array([-196.58, -196.58,  -30.  ]), array([196.58, 196.58, -30.  ]), array([-196.58,  196.58,  -30.  ])]


In [6]:
# reveal how the sm is placed in the room
print('true:', sm.get_placement('true'))

# initialize the estimated placement with the design value
sm.place_est = sm.place_design.copy()
print('est:', sm.get_placement('est'))

true: {'location': [-19.21223793079177, -44.750123502954, -49.07459555657978], 'direction_x': [0.9992820744861418, -0.021455536496225624, -0.031224918964369085], 'direction_z': [0.031146403128285035, -0.00398402560533684, 0.9995068929788058]}
est: {'location': [0.0, 0.0, 0.0], 'direction_x': [1.0, 0.0, 0.0], 'direction_z': [0.0, 0.0, 1.0]}


In [7]:
# Simulate survey data by smearing the true positions of the fiducial markers
# later try adding some data entry errors!

survey_std = 0.5 # mm

survey_sm_data = []
for mpmt in sm.mpmts:
    survey_mpmt_data = []
    fiducials_true = mpmt.get_fiducials('true')
    for fiducial_true in fiducials_true:
        survey_mpmt_data.append(np.add(fiducial_true, stats.norm.rvs(0., survey_std, size=3)))
    survey_sm_data.append(survey_mpmt_data)


In [8]:
# Combine the survey data to deduce the placement of the sm in the room (placement defined by 6 parameters: 3 locations and 3 rotations)
# then use the survey data for each mpmt to deduce its placement within the sm

# since some of the fiducials might not be measured in the actual survey, do not use their average for an mpmt to characterize the mpmt location
# instead, calculate the RMS of all fiducials: first assuming mpmt are placed according to design (to find sm placement) then with sm placement do each mpmt
# check if an additional iteration helps

# Start with model that sets all estimated placements (for the sm wrt room and the mpmts wrt sm) to their design values: execute this to start over

sm.place_est = sm.place_design.copy()
for mpmt in sm.mpmts:
    mpmt.place_est = mpmt.place_design.copy()

In [9]:
# Calculate sum of squares of residuals for all fiducials in sm 

def calc_dev2_all():
    sum2 = 0.
    for i_mpmt, mpmt in enumerate(sm.mpmts):
        fiducials_est = mpmt.get_fiducials('est')
        for i in range(4):
            diff = np.subtract(fiducials_est[i],survey_sm_data[i_mpmt][i])
            sum2 += np.dot(diff,diff)
    return sum2

# Calculate sum of squares of residuals for the fiducials in the fit mpmt

fit_mpmt = 0

def calc_dev2_one():
    sum2 = 0.
    fiducials_est = sm.mpmts[fit_mpmt].get_fiducials('est')
    for i in range(4):
        diff = np.subtract(fiducials_est[i],survey_sm_data[fit_mpmt][i])
        sum2 += np.dot(diff,diff)
    return sum2

# parameters specify the sm placement
def calc_dev2_sm(params):
    loc = [params[i] for i in range(3)]
    rot_angles = [params[i] for i in range(3,6)]
    sm.place_est['loc'] = loc
    sm.place_est['rot_angles'] = rot_angles
    
    return calc_dev2_all()

# parameters specify the fit_mpmt placement
def calc_dev2_mpmt(params):
    loc = [params[i] for i in range(3)]
    rot_angles = [params[i] for i in range(3,6)]
    sm.mpmts[fit_mpmt].place_est['loc'] = loc
    sm.mpmts[fit_mpmt].place_est['rot_angles'] = rot_angles
    
    return calc_dev2_one()


In [10]:
sum2 = calc_dev2_all()
print('all:',np.sqrt(sum2/(4*len(sm.mpmts))))

sum2 = calc_dev2_one()
print('one:',np.sqrt(sum2/4))

all: 77.80574193556808
one: 66.55048174180155


In [11]:
# find optimal sm placement parameters - start from the current values stored in place_est
start = sm.place_est['loc'] + sm.place_est['rot_angles']
result = optimize.minimize(calc_dev2_sm, start)
print(result.message)
    
print('true:', sm.get_placement('true'))
print('est:', sm.get_placement('est'))

sum2 = calc_dev2_all()
print('rms of residuals:', np.sqrt(sum2/(4*len(sm.mpmts))))

Desired error not necessarily achieved due to precision loss.
true: {'location': [-19.21223793079177, -44.750123502954, -49.07459555657978], 'direction_x': [0.9992820744861418, -0.021455536496225624, -0.031224918964369085], 'direction_z': [0.031146403128285035, -0.00398402560533684, 0.9995068929788058]}
est: {'location': [-18.91597019146774, -44.163158529835066, -49.8990715910163], 'direction_x': [0.9992620773608463, -0.022049051155929908, -0.031450597635168505], 'direction_z': [0.03138710842528338, -0.0032197197144078144, 0.9995021174713239]}
rms of residuals: 8.786280164765344


In [12]:
# find optimal mpmt placement parameters - start from the current values stored in place_est

for fit_mpmt in range(len(sm.mpmts)):

    start = list(sm.mpmts[fit_mpmt].place_est['loc']) + list(sm.mpmts[fit_mpmt].place_est['rot_angles'])
    result = optimize.minimize(calc_dev2_mpmt, start)
    
    rms = np.sqrt(calc_dev2_one()/4.)
    if rms > 1.:
        print('mpmt', fit_mpmt, result.message)
            
        print('true:', sm.mpmts[fit_mpmt].get_placement('true'))
        print('est:', sm.mpmts[fit_mpmt].get_placement('est'))
        
        print('rms of residuals:', rms)
        
sum2 = calc_dev2_all()
print('rms all:',np.sqrt(sum2/(4*len(sm.mpmts))))

rms all: 0.6565746867441148


In [11]:
# Show in 3D the deviations between est and survey (or between est and true) to illustrate how survey optimization proceeds 

# to keep mpmt numbering consistent with the full WCTE detector, we need to add an offset
first_mpmt = {'bottom':0, 'barrel':21, 'top':85}[sm.name]

plot = k3d.plot()


origins = []
survey_vecs = []

survey_scale = 1. # scale up the length of deviations
color_survey = 0xff0000

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

n_point_ft = 20 # for feedthroughs and C holes
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])

# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(sm.mpmts):
    
    p = mpmt.get_placement('est')
    location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']

    baseplate_points = np.array(mpmt.get_xy_points('est'),dtype=np.float32)
    plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_baseplate

    feedthrough_points = np.array(mpmt.get_xy_points('est', feature='feedthrough'),dtype=np.float32)
    plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
    plot += plt_feedthrough

    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_mpmt+first_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    plot += plt_text

    # draw the fiducial points
    fiducial_points = np.array(mpmt.get_fiducials('est'),dtype=np.float32)
    for i_f, fiducial_point in enumerate(fiducial_points):
        text_location = [fiducial_point[j] - 10.*direction_z[j] for j in range(3)]
        plt_text = k3d.text('f'+str(i_f+1), position=text_location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
        plot += plt_text

    fiducial_locations = np.array(fiducial_points, dtype=np.float32)
    plt_fps = k3d.points(positions=fiducial_locations,
                        point_size=8.,
                        shader='3d',
                        color=color_mpmt)
    plot += plt_fps

    # show the surveyed locations of the fiducial points
    fiducial_survey = np.array(survey_sm_data[i_mpmt], dtype=np.float32)
    plt_fs = k3d.points(positions=fiducial_survey,
                        point_size=8.,
                        shader='3d',
                        color=color_survey)
    plot += plt_fs
    
    # show a vector from the estimated to the surveyed
    fiducials_est = mpmt.get_fiducials('est')
    for i in range(4):
        origins.append(fiducials_est[i])
        diff = np.subtract(survey_sm_data[i_mpmt][i],fiducials_est[i])
        scaled_diff = list(diff*survey_scale)
        survey_vecs.append(scaled_diff)



plt_survey_vecs = k3d.vectors(origins=origins, vectors=survey_vecs, color=color_survey, head_size=250.)
plot += plt_survey_vecs

plot.display()

C:\Users\Karlen\AppData\Local\Programs\Python\Python39\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [20]:
# save the top level device (and thereby the entire geometry)
my_room.save_file('my_room.geo')

In [21]:
# read the saved device back again
from Geometry.Device import Device
new_room = Device.open_file('my_room.geo')

In [22]:
new_room.sms[0].mpmts[2].get_placement('est')

{'location': array([763.17552018, 577.1726509 , -48.17788806]),
 'direction_x': array([ 0.03481244, -0.99861181,  0.03952902]),
 'direction_z': array([0.06384471, 0.0416944 , 0.99708848])}